In [34]:
import pandas as pd
import requests
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
from datetime import datetime, timedelta
import random
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
model_id = "deepseek-ai/deepseek-llm-7b-chat"
torch.cuda.set_device(3)  # Sets default to GPU 0
device=torch.device("cuda:3")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": 3},             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)

/home/f20222001/test-venv/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:  50%|█████     | 1/2 [00:19<00:19, 19.18s/it]


KeyboardInterrupt: 

In [32]:
class fundamental_analyst:

    def __init__ (self, ticker):
        self.ticker=ticker
        self.get_financial_info()
        self.generate_prompt()
        
    def get_financial_info(self):
        ticker = yf.Ticker(self.ticker)
        self.income_stmt = ticker.financials
        self.balance_sheet = ticker.balance_sheet
        self.cash_flow = ticker.cashflow
        self.info = ticker.info
        self.year=self.income_stmt.columns[0]
        self.EPS=self.income_stmt[self.year]["Diluted EPS"]
        self.Net_Income=self.income_stmt[self.year]["Net Income"]
        self.Gross_Profit=self.income_stmt[self.year]["Gross Profit"]
        self.Revenue=self.income_stmt[self.year]["Total Revenue"]
        self.Total_Assets=self.balance_sheet[self.year]["Total Assets"]
        self.Total_Liabilities=self.balance_sheet[self.year]["Total Liabilities Net Minority Interest"]
        self.Shareholders_Equity=self.balance_sheet[self.year]["Stockholders Equity"]
        self.Free_Cash_Flow=self.cash_flow[self.year]["Free Cash Flow"]
        self.Investing_Cash_Flow=self.cash_flow[self.year]["Investing Cash Flow"]
        self.Financing_Cash_Flow=self.cash_flow[self.year]["Financing Cash Flow"]
        self.Operating_Cash_Flow=self.cash_flow[self.year]["Operating Cash Flow"]
        self.P_E=ticker.info['trailingPE']
        self.ROA=ticker.info['returnOnAssets']
        self.ROE=ticker.info['returnOnEquity']
    
    def generate_prompt(self):
        prompt = f"""
Pretend that you are a fundamental investment analyst. Analyze the financial performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
Do not add assumptions about company operations, reputation, or strategy.

Financials for {self.ticker}:

Income Statement:
Revenue: ${self.Revenue:,.0f}
Gross Profit: ${self.Gross_Profit:,.0f}
Net Income: ${self.Net_Income:,.0f}
EPS (Diluted): {self.EPS:.2f}

Balance Sheet:
Total Assets: ${self.Total_Assets:,.0f}
Total Liabilities: ${self.Total_Liabilities:,.0f}
Shareholders' Equity: ${self.Shareholders_Equity:,.0f}

Cash Flow:
Operating Cash Flow: ${self.Operating_Cash_Flow:,.0f}
Free Cash Flow: ${self.Free_Cash_Flow:,.0f}
Investing Cash Flow: ${self.Investing_Cash_Flow:,.0f}
Financing Cash Flow: ${self.Financing_Cash_Flow:,.0f}

Valuation and Ratios:
P/E Ratio: {self.P_E:.2f}
ROA: {self.ROA:.2%}
ROE: {self.ROE:.2%}

Based on this, what is your investment recommendation? Pick one action candidate."""
        self.prompt=prompt
        return self.prompt
        
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )
        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only

In [9]:
class technical_analyst:

    def __init__(self, ticker):
        self.ticker=ticker
        self.generate_df()
        self.generate_indicators()
        self.generate_technical_prompt()
        
    def generate_df(self):
        self.data=yf.download(self.ticker, period='1y')

    def compute_rsi(self, close, period=14):
        delta = close.diff()

        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)

        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def compute_obv(self, close, volume):
        direction = close.diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
        obv = (volume * direction).fillna(0).cumsum()
        return obv
    
    def generate_indicators(self):
        temp=pd.DataFrame()
        temp["SMA_5"] = self.data['Close'][self.ticker].rolling(5).mean()
        temp["SMA_15"] = self.data['Close'][self.ticker].rolling(15).mean()
        temp["SMA_50"] = self.data['Close'][self.ticker].rolling(50).mean()

        temp['EMA_5'] = self.data['Close'][self.ticker].ewm(span=5).mean()
        temp['EMA_10'] = self.data['Close'][self.ticker].ewm(span=10).mean()
        temp['EMA_50'] = self.data['Close'][self.ticker].ewm(span=50).mean()
        temp["Date"] = self.data['Close'][self.ticker].index
        temp["RSI"]=self.compute_rsi(self.data['Close'][self.ticker])
        temp["OBV"]=self.compute_obv(self.data['Close'][self.ticker], self.data['Volume'][self.ticker])
        self.indicator_df=temp

    def generate_technical_prompt(self):
        latest = self.indicator_df.iloc[-1]
        prompt = f"""
ou are a technical investment analyst. Analyze the recent technical performance of {self.ticker} and give an investment recommendation.

Your task:
Choose one of the following recommendations: Strong Buy, Buy, Hold, Sell, or Short.
Justify your choice using 4–6 bullet points based only on the indicators below.
You MUST respond in natural human language. Do NOT include any code or formulas.

Technical Indicators for {self.ticker} (most recent data point):

SMA 5: {latest['SMA_5']:.2f}
SMA 15: {latest['SMA_15']:.2f}
SMA 50: {latest['SMA_50']:.2f}

EMA 5: {latest['EMA_5']:.2f}
EMA 10: {latest['EMA_10']:.2f}
EMA 50: {latest['EMA_50']:.2f}

RSI: {latest['RSI']:.2f}
OBV: {latest['OBV']:,.0f}

Based on this, what is your investment recommendation? Pick one action candidate.
"""
        self.prompt=prompt
        return self.prompt 
    
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=1000,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only


In [ ]:
class news_analyst:

    def __init__ (self,ticker, company):
        self.ticker=ticker
        self.company=company
        self.get_news_articles()
        self.generate_news_prompt()
        
    def get_news_articles(self):
        to_date = datetime.today().date()

        from_date = to_date - timedelta(days=7)

        from_str = from_date.strftime('%Y-%m-%d')
        to_str = to_date.strftime('%Y-%m-%d')

        symbol = self.ticker
        api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
        url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={"2015-07-01"}&to={"2016-07-01"}&token={api_key}"
        response = requests.get(url)
        news = response.json()
        filtered_news = [
            item for item in news
            if self.company in item['headline'].lower()
        ]
        self.selected_news = random.sample(filtered_news, k=min(10, len(filtered_news)))
    
    def generate_news_prompt(self):
        prompt = f"""
Pretend that you are a sentiment and headlines investment analyst. Analyze the recent technical performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using sentiment analysis. Only use the headlines given. 
Do not add assumptions about company fundamentals, operations, or strategy.

Headlines:
"""
        for i, item in enumerate(self.selected_news):
            prompt+=item['headline']
            prompt+='\n'
        prompt+="Based on this, what is your investment recommendation? Pick one action candidate."
        self.prompt=prompt
        return self.prompt
    
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )

        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only

In [1]:
import requests
api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
url = f"https://finnhub.io/api/v1/stock/financials-reported?symbol=AAPL&token={api_key}"
response = requests.get(url)
dats=response.json()


In [2]:
dats.keys()

dict_keys(['cik', 'data', 'symbol'])

In [8]:
for dic in dats['data']:
    if dic['year']==2021:
        print(dic)


{'accessNumber': '0000320193-21-000105', 'symbol': 'AAPL', 'cik': '320193', 'year': 2021, 'quarter': 0, 'form': '10-K', 'startDate': '2020-09-27 00:00:00', 'endDate': '2021-09-25 00:00:00', 'filedDate': '2021-10-29 00:00:00', 'acceptedDate': '2021-10-28 18:04:28', 'report': {'bs': [{'concept': 'us-gaap_CashAndCashEquivalentsAtCarryingValue', 'unit': 'usd', 'label': 'Cash and cash equivalents', 'value': 34940000000.0}, {'concept': 'us-gaap_MarketableSecuritiesCurrent', 'unit': 'usd', 'label': 'Marketable securities', 'value': 27699000000.0}, {'concept': 'us-gaap_AccountsReceivableNetCurrent', 'unit': 'usd', 'label': 'Accounts receivable, net', 'value': 26278000000}, {'concept': 'us-gaap_InventoryNet', 'unit': 'usd', 'label': 'Inventories', 'value': 6580000000}, {'concept': 'us-gaap_NontradeReceivablesCurrent', 'unit': 'usd', 'label': 'Vendor non-trade receivables', 'value': 25228000000}, {'concept': 'us-gaap_OtherAssetsCurrent', 'unit': 'usd', 'label': 'Other current assets', 'value': 1

In [14]:
dats['data'][4]['report']['ic']

[{'concept': 'us-gaap_RevenueFromContractWithCustomerExcludingAssessedTax',
  'unit': 'usd',
  'label': 'Net sales',
  'value': 274515000000.0},
 {'concept': 'us-gaap_CostOfGoodsAndServicesSold',
  'unit': 'usd',
  'label': 'Cost of sales',
  'value': 169559000000.0},
 {'concept': 'us-gaap_GrossProfit',
  'unit': 'usd',
  'label': 'Gross margin',
  'value': 104956000000},
 {'concept': 'us-gaap_ResearchAndDevelopmentExpense',
  'unit': 'usd',
  'label': 'Research and development',
  'value': 18752000000},
 {'concept': 'us-gaap_SellingGeneralAndAdministrativeExpense',
  'unit': 'usd',
  'label': 'Selling, general and administrative',
  'value': 19916000000},
 {'concept': 'us-gaap_OperatingExpenses',
  'unit': 'usd',
  'label': 'Total operating expenses',
  'value': 38668000000},
 {'concept': 'us-gaap_OperatingIncomeLoss',
  'unit': 'usd',
  'label': 'Operating income',
  'value': 66288000000.0},
 {'concept': 'us-gaap_NonoperatingIncomeExpense',
  'unit': 'usd',
  'label': 'Other income/(

In [15]:
for dic in dats['data']:
    if dic['year']==2021 :
        for small in dic['report']['ic']:
            if 'us-gaap_NetIncomeLoss' in small.values():
                print(small)

{'concept': 'us-gaap_NetIncomeLoss', 'unit': 'usd', 'label': 'Net income', 'value': 94680000000.0}


In [ ]:
def find_us_gaap_entry(dats, parameter, year):
    for dic in dats['data']:
        if dic.get('year') == year:
            for entry in dic.get('report', {}).get('ic', []):
                if parameter in entry.values():
                    return entry
    return None

In [ ]:
ic_vals=['us-gaap_NetIncomeLoss','us-gaap_EarningsPerShareDiluted','us-gaap_LongTermDebtNoncurrent',\
         'us-gaap_CashAndCashEquivalentsAtCarryingValue']

AttributeError: 'fundamental_analyst' object has no attribute 'gaap_data'

In [13]:
# techni=technical_analyst('NVDA')
# print(techni.prompt)
# techni_analysis=techni.generate_response()
# print("Response:")
# print(techni_analysis)

In [14]:
# news=news_analyst('NVDA','nvidia')
# print(news.prompt)
# news_analysis=news.generate_response()
# print("Response:")
# print(news_analysis)

In [16]:
import requests
api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
url = f"https://finnhub.io/api/v1/stock/metric?symbol=AAPL&token={api_key}"
response = requests.get(url)
dat=response.json()


In [ ]:
def find_metric_by_year(dat, parameter, year):
    series = dat.get('series', {}).get('annual', {}).get(parameter, [])
    for item in series:
        if item.get('period', '').startswith(str(year)):
            return item
    return None 


In [23]:
for item in dat['series']['annual']['pe']:
    if item['period'].startsWith('2024'):
        print (item)

    

[{'period': '2024-09-28', 'v': 37.5759},
 {'period': '2023-09-30', 'v': 27.8553},
 {'period': '2022-09-24', 'v': 24.0854},
 {'period': '2021-09-25', 'v': 25.3801},
 {'period': '2020-09-26', 'v': 34.2462},
 {'period': '2019-09-28', 'v': 18.3176},
 {'period': '2018-09-29', 'v': 18.4383},
 {'period': '2017-09-30', 'v': 16.4312},
 {'period': '2016-09-24', 'v': 13.3133},
 {'period': '2015-09-26', 'v': 11.748},
 {'period': '2014-09-27', 'v': 14.8636},
 {'period': '2013-09-28', 'v': 11.6944},
 {'period': '2012-09-29', 'v': 14.8112},
 {'period': '2011-09-24', 'v': 14.4532},
 {'period': '2010-09-25', 'v': 18.9822},
 {'period': '2009-09-26', 'v': 20.2499},
 {'period': '2008-09-27', 'v': 15.239},
 {'period': '2007-09-29', 'v': 38.901},
 {'period': '2006-09-30', 'v': 32.0106},
 {'period': '2005-09-24', 'v': 33.6431},
 {'period': '2004-09-25', 'v': 54.7323},
 {'period': '2003-09-27', 'v': 111.9025},
 {'period': '2002-09-28', 'v': 80.0592},
 {'period': '2001-09-29', 'v': None},
 {'period': '2000-09-

In [ ]:
ratios=['grossMargin','operatingMargin','roe','currentRatio','quickRatio','totalDebttoEquity','pe','pb','ev','salesPerShare']

In [36]:
class fundamental_analyst:

    def __init__(self, ticker, years: list):
        self.ticker = ticker
        self.years = years
        self.get_data()
        self.get_financial_info()
        self.generate_prompt()
        
    def get_data(self):
        api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
        url = f"https://finnhub.io/api/v1/stock/metric?symbol=AAPL&token={api_key}"
        response = requests.get(url)
        self.metric_data=response.json()
        url = f"https://finnhub.io/api/v1/stock/financials-reported?symbol=AAPL&token={api_key}"
        response = requests.get(url)
        self.gaap_data=response.json()

    def find_us_gaap_entry(self, parameter, year):
        for dic in self.gaap_data['data']:
            if dic.get('year') == year:
                for section in ['ic', 'bs', 'cf']:  # income statement, balance sheet, cash flow
                    for entry in dic.get('report', {}).get(section, []):
                        if parameter in entry.values():
                            return entry.get('value')
        return None

    def find_metric_by_year(self, parameter, year):
        series = self.metric_data.get('series', {}).get('annual', {}).get(parameter, [])
        for item in series:
            if item.get('period', '').startswith(str(year)):
                return item.get('v')  # assuming value is under 'v'
        return None

    def get_financial_info(self):
        self.EPS = []
        self.Net_Income = []
        self.Gross_Profit = []
        self.Revenue = []
        self.Total_Assets = []
        self.Total_Liabilities = []
        self.Shareholders_Equity = []
        self.Free_Cash_Flow = []
        self.Operating_Cash_Flow = []
        self.Investing_Cash_Flow = []
        self.Financing_Cash_Flow = []
        self.P_E = []
        self.ROA = []
        self.ROE = []

        for year in self.years:
            self.EPS.append(self.find_us_gaap_entry('us-gaap_EarningsPerShareDiluted', year))
            self.Net_Income.append(self.find_us_gaap_entry('us-gaap_NetIncomeLoss', year))
            self.Gross_Profit.append(self.find_us_gaap_entry('us-gaap_GrossProfit', year))
            self.Revenue.append(self.find_us_gaap_entry('us-gaap_RevenueFromContractWithCustomerExcludingAssessedTax', year))
            self.Total_Assets.append(self.find_us_gaap_entry('us-gaap_Assets', year))
            self.Total_Liabilities.append(self.find_us_gaap_entry('us-gaap_Liabilities', year))
            self.Shareholders_Equity.append(self.find_us_gaap_entry('us-gaap_StockholdersEquity', year))
            self.Operating_Cash_Flow.append(self.find_us_gaap_entry('us-gaap_NetCashProvidedByUsedInOperatingActivities', year))
            self.Investing_Cash_Flow.append(self.find_us_gaap_entry('us-gaap_NetCashProvidedByUsedInInvestingActivities', year))
            self.Financing_Cash_Flow.append(self.find_us_gaap_entry('us-gaap_NetCashProvidedByUsedInFinancingActivities', year))
            self.P_E.append(self.find_metric_by_year('pe', year))
            self.ROA.append(self.find_metric_by_year('roa', year))
            self.ROE.append(self.find_metric_by_year('roe', year))

    
    def generate_prompt(self):
        prompt = f"""
    Pretend that you are a fundamental investment analyst. Analyze the financial performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
    Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
    Do not add assumptions about company operations, reputation, or strategy.

    Financials for {self.ticker}:\n
    """
        for i, year in enumerate(self.years):
            prompt += f"""
    Year: {year}
    Income Statement:
    Revenue: ${self.Revenue[i]:,.0f}
    Gross Profit: ${self.Gross_Profit[i]:,.0f}
    Net Income: ${self.Net_Income[i]:,.0f}
    EPS (Diluted): {self.EPS[i]:.2f}

    Balance Sheet:
    Total Assets: ${self.Total_Assets[i]:,.0f}
    Total Liabilities: ${self.Total_Liabilities[i]:,.0f}
    Shareholders' Equity: ${self.Shareholders_Equity[i]:,.0f}

    Cash Flow:
    Operating Cash Flow: ${self.Operating_Cash_Flow[i]:,.0f}
    Investing Cash Flow: ${self.Investing_Cash_Flow[i]:,.0f}
    Financing Cash Flow: ${self.Financing_Cash_Flow[i]:,.0f}

    Valuation and Ratios:
    P/E Ratio: {self.P_E[i]:.2f}
    ROA: {self.ROA[i]:.2%}
    ROE: {self.ROE[i]:.2%}

    """
        prompt += "\nBased on this, what is your investment recommendation? Pick one action candidate."
        self.prompt = prompt
        return prompt

        
    def generate_response(self):
        generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer
        )
        outputs = generator(
            self.prompt,
            max_new_tokens=500,         # Reduced for memory efficiency
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            # Memory efficient generation settings
            num_beams=1,                # No beam search to save memory
            #early_stopping=True,
            use_cache=True
        )

        full_text = outputs[0]['generated_text']

        response_only = full_text[len(self.prompt):].strip()
        return response_only

In [37]:
funda=fundamental_analyst('NVDA',years=[2023,2024])
print(funda.prompt)
funda_analysis=funda.generate_response()
print("Response")
print(funda_analysis)


    Pretend that you are a fundamental investment analyst. Analyze the financial performance of NVDA and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
    Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
    Do not add assumptions about company operations, reputation, or strategy.

    Financials for NVDA:

    
    Year: 2023
    Income Statement:
    Revenue: $383,285,000,000
    Gross Profit: $169,148,000,000
    Net Income: $96,995,000,000
    EPS (Diluted): 6.13

    Balance Sheet:
    Total Assets: $352,583,000,000
    Total Liabilities: $290,437,000,000
    Shareholders' Equity: $62,146,000,000

    Cash Flow:
    Operating Cash Flow: $110,543,000,000
    Investing Cash Flow: $3,705,000,000
    Financing Cash Flow: $-108,488,000,000

    Valuation and Ratios:
    P/E Ratio: 27.86
    ROA: 27.51%
    ROE: 156.08%

    
    Year: 2024
    Income Statement:
    

NameError: name 'model' is not defined